In [23]:
import pandas as pd
import numpy as np
from usfull_tools import reduce_mem_usage
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
%matplotlib inline

from set_vars import KAGGLE_PREFIX, debug_mode, KAGGLE_DIR, target_column, target_type='binary'


if debug_mode == True:
    train = pd.read_csv(KAGGLE_DIR + 'train' + KAGGLE_PREFIX + '_prepare.csv', nrows=100000)
    test = pd.read_csv(KAGGLE_DIR + 'test' + KAGGLE_PREFIX + '_prepare.csv', nrows=100000)
else:
    train = pd.read_csv(KAGGLE_DIR + 'train' + KAGGLE_PREFIX + '_prepare.csv')
    test = pd.read_csv(KAGGLE_DIR + 'test' + KAGGLE_PREFIX + '_prepare.csv')

train = reduce_mem_usage(train)

X_train, X_test, y_train, y_test = train_test_split(train[train.columns.drop(target_column)], train[target_column], 
                                                    test_size=0.3, random_state=42)
X_train.shape

(623, 490)

In [26]:
# Column prefixes:
# _nan - original columns with Nan
# _nan_median, _nan_min, _nan_max, _nan0 - nan replaced by median
# _idxmax - nan replaced by most frequent
# _isnull - is null flag
# _dummie - dumnie for categorical column

i=0
cat_features = []
for column in X_train.columns:
    if X_train[column].dtype == 'object': cat_features.append(i)
    i +=1

    
# print(cat_features)
# train_pool = Pool(X_train, y_train, cat_features)
# test_pool = Pool(X_test, cat_features)

if target_type == 'binary': 
    loss_function = 'Logloss:border=0.5'
    clf = CatBoostClassifier(iterations=60, depth=2, learning_rate=0.1, loss_function=loss_function)

#https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_parameters-list-docpage/#python-reference_parameters-list
#clf = CatBoostRegressor(iterations=60, depth=2, learning_rate=0.1, loss_function=loss_function)

!!! Ошибка возникает из-за nan в категориальных столбцах, надо его заменить на текстовое 'NAN' в процессе преодработки
clf.fit(X_train, y_train, cat_features)

# from sklearn.metrics import mean_absolute_error, accuracy
# print("accuracy: %.3f"
#       % accuracy(clf.predict(test_pool), y_test))

# submit['winPlacePercPredict'] = clf.predict(submit[used_cols])
# submit['matchRank'] = submit.groupby('matchId')['winPlacePercPredict'].rank()
# match_players = submit.groupby('matchId')['Id'].count()
# match_step = 1/(match_players+1)

# match_stat = pd.DataFrame({'matchId': match_players.index,
#                            'players': match_players.values,
#                            'matchStep': match_step.values})

# sub = pd.merge(submit, match_stat, on='matchId',how='inner')
# sub.sort_values(['matchId', 'winPlacePercPredict'], inplace=True)
# sub['winPlacePerc'] = (sub['matchRank']-1)*sub['matchStep']
# sub.loc[sub.isnull().any(axis=1), ['winPlacePerc']]=1


# sub.to_csv('submission.csv', columns=['Id', 'winPlacePerc'], index=False, sep=',', decimal='.')

CatboostError: Invalid type for cat_feature[1,16]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.